# Self-Driving Car Engineer Nanodegree


## Project: **Advanced Lane Lines** 
***
### The Goal of this Project
In this project, your goal is to write a software pipeline to identify the lane boundaries in a video from a front-facing camera on a car. The camera calibration images, test road images, and project videos are available in the [project repository](https://github.com/udacity/CarND-Advanced-Lane-Lines).

### The steps of this project are the following:

1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
2. Apply a distortion correction to raw images.
3. Apply a perspective transform to rectify distortion corrected image ("birds-eye view").
4. Use color threshold and/or gradient transforms to create a binary image.
5. Detect lane pixels and fit to find the lane boundary.
6. Determine the radius of curvature and car offset.
7. Warp the detected lane boundaries back onto the original image.
8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
9. Line fit based on sanity checks.
10. Integrate the pipeline and test on images.
11. Test on videos.



The images for camera calibration are stored in the folder called camera_cal. The images in test_images are for testing your pipeline on single frames. If you want to extract more test images from the videos, you can simply use an image writing method like cv2.imwrite(), i.e., you can read the video in frame by frame as usual, and for frames you want to save for later you can write to an image file.

To help the reviewer examine your work, please save examples of the output from each stage of your pipeline in the folder called ouput_images, and include a description in your writeup for the project of what each image shows. The video called project_video.mp4 is the video your pipeline should work well on.

The challenge_video.mp4 video is an extra (and optional) challenge for you if you want to test your pipeline under somewhat trickier conditions. The harder_challenge.mp4 video is another optional challenge and is brutal!

If you're feeling ambitious (again, totally optional though), don't stop there! We encourage you to go out and take video of your own, calibrate your camera and show us how you would implement this project from scratch!

### Step-1: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

(i) Use chessboard images and OpenCV functions [findChessboardCorners()](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.findChessboardCorners) and [drawChessboardCorners()](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.drawChessboardCorners) to [extract image points and object points](https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb)
(ii) Use the OpenCV functions [cv2.calibrateCamera()](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_calibration/py_calibration.html) to calculate camera calibration matrix and distortion coefficients.

In [ ]:
# Imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

In [ ]:
# Visualize Results
def visualize_results(img1, img2, str1, str2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(str1, fontsize=30)
    ax2.imshow(img2, cmap='gray')
    ax2.set_title(str2, fontsize=30)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Compute the camera calibration matrix and distortion coefficients
def compute_camera(camera_cal_images, init_img, nx, ny):
    # extract object points and image points for camera calibration
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    #images = glob.glob('camera_cal/calib*.jpg')
    images = glob.glob(camera_cal_images)
    
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.copy(img_rgb)
        # Find the chessboard corners
        #ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        ret, corners = cv2.findChessboardCorners(img_rgb, (nx,ny), None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            img_o = cv2.drawChessboardCorners(img_rgb, (nx,ny), corners, ret)
            #write_name = 'corners_found'+str(idx)+'.jpg'
            #cv2.imwrite(write_name, img)
            #cv2.imshow('img', img)
            #cv2.waitKey(1000)
            visualize_results(img, img_o, 'Origial Image', 'Image Corners')
    #cv2.destroyAllWindows()
    init_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    init_img_size = (init_img.shape[1], init_img.shape[0])
    #print("image size=", img_size)
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, init_img_size, None, None)
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump(dist_pickle, open("output_images/wide_dist_pickle.p", "wb" ) )
    #print("Camera calib. is done (one time only)")

In [ ]:
# Test compute_camera 
%matplotlib inline
nx  = 9
ny = 6
camera_cal_images = 'camera_cal/calib*.jpg'
init_img = mpimg.imread('test_images/straight_lines1.jpg')
compute_camera(camera_cal_images, init_img, nx, ny)

### Step-2: Apply a distortion correction to raw images.

Use the OpenCV functions [cv2.calibrateCamera()](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_calibration/py_calibration.html) and [cv2.undistort()](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_calibration/py_calibration.html) to compute the calibration and undistortion.

In [ ]:
# Distortion Correction
def dist_correct(img):
    # Read in the saved camera matrix and distortion coefficients
    # These are the arrays you calculated using cv2.calibrateCamera()
    dist_pickle = pickle.load(open("output_images/wide_dist_pickle.p" , "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    # undistort
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    undist = cv2.cvtColor(undist, cv2.COLOR_BGR2RGB)
    #print("1-Camera distortion correction is done...")
    return undist

In [ ]:
# Test dist_correct on checker board images
t_images = sorted(glob.glob('camera_cal/*.jpg'))
for idx, pname,  in enumerate(t_images):
    fname = pname.split('/')[-1]
    # Read in an image
    img = cv2.imread(pname)
    #test_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Run the function
    undist = dist_correct(img)
    # Save and visualize output images
    cv2.imwrite('output_images/'+'output_'+fname, undist)
    visualize_results(img, undist, pname, 'output_images/'+fname)

In [ ]:
# Test dist_correct on all test images
t_images = sorted(glob.glob('test_images/*.jpg'))
for idx, pname,  in enumerate(t_images):
    fname = pname.split('/')[-1]
    # Read in an image
    img = cv2.imread(pname)
    test_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Run the function
    undist = dist_correct(img)
    # Save and/or visualize output images
    cv2.imwrite('output_images/'+'output_'+fname, undist)
    visualize_results(test_image, undist, pname, 'output_images/'+fname )

### Step-3: Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
# Read source image points using interactive window
#%matplotlib qt5 
%matplotlib inline 
image = cv2.imread('test_images/straight_lines1.jpg')
test_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.subplots(1, 1, figsize=(10,5))
plt.imshow(test_image)
# source image points (read from interactive window)
plt.plot(720, 470, ".") # top right
plt.plot(1031, 670, ".") # bottom right
plt.plot(275, 670, ".") # bottom left
plt.plot(569, 470, ".") # top left

In [ ]:
# Perspective Transform 
def corners_unwarp(img):
    
    img_size = (img.shape[1], img.shape[0])

    # source points
    src = np.float32([[720, 470],[1031, 670],[275, 670],[569, 470]])
    # destination points
    dst = np.float32([[1031,100],[1031,670],[275,670],[275,100]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src) 
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    #print("Perspective transform is done...")
    return warped, M, Minv

In [ ]:
# Test Perspective Transform
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    birds_eye, M, Minv = corners_unwarp(img)
    # Plot the result
    visualize_results(img, birds_eye, pname, 'Bird\'s Eye Image%s'%idx)

### Step-4: Use color threshold and/or gradient transforms to create a binary image.

### Exploring Color Thresholds

In [ ]:
# R of RGB 
def r_color_thresh(img, r_thresh=(0, 255)):
    r_channel = img[:,:,0]
    r_binary_output = np.zeros_like(r_channel)
    r_binary_output[(r_channel > r_thresh[0]) & (r_channel <= r_thresh[1])] = 1
    return r_binary_output

In [ ]:
# Test of RGB
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    r_binary_output = r_color_thresh(img, r_thresh=(215, 255))
    # Plot the result
    #visualize_results(img, r_binary_output, pname, 'R Color Thresh.%s'%idx)

In [ ]:
# V of HSV
def v_color_thresh(img, v_thresh=(0, 255)):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary_output = np.zeros_like(v_channel)
    v_binary_output[(v_channel > v_thresh[0]) & (v_channel <= v_thresh[1])] = 1
    return v_binary_output

In [ ]:
# Test of HSV
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    v_binary_output = v_color_thresh(img, v_thresh=(220, 255))
    # Plot the result
    #visualize_results(img, v_binary_output, pname, 'V Color Thresh.%s'%idx)

In [ ]:
# L of LUV
def l_color_thresh(img, l_thresh=(0, 255)):
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    l_channel = luv[:,:,0]
    l_binary_output = np.zeros_like(l_channel)
    l_binary_output[(l_channel > l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    return l_binary_output

In [ ]:
# Test LUV
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    l_binary_output = l_color_thresh(img, l_thresh=(210, 255))
    # Plot the result
    #visualize_results(img, l_binary_output, pname, 'L Color Thresh.%s'%idx)

In [ ]:
# B of LAB 
def b_color_thresh(img, b_thresh=(0, 255)):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    b_channel = lab[:,:,2]
    b_binary_output = np.zeros_like(b_channel)
    b_binary_output[(b_channel > b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    return b_binary_output

In [ ]:
# Test LAB
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    b_binary_output = b_color_thresh(img, b_thresh=(150, 255))
    # Plot the result
    #visualize_results(img, b_binary_output, pname, 'B Color Thresh.%s'%idx)

In [ ]:
# Combined Color Threshold
def comb_color_thresh(img):
    r_binary_output = r_color_thresh(img, r_thresh=(215,255))
    v_binary_output = v_color_thresh(img, v_thresh=(220,255))
    l_binary_output = l_color_thresh(img, l_thresh=(210,255))
    b_binary_output = b_color_thresh(img, b_thresh=(150,255))
    binary_output = np.zeros_like(l_binary_output)
    binary_output[(r_binary_output == 1) | (v_binary_output == 1) | (l_binary_output == 1) | (b_binary_output == 1)] = 1
    
    #print("color transform is done...")
    return binary_output

In [ ]:
# Test combined color threshold
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    birds_eye, M, Minv = corners_unwarp(img)
    binary_color = comb_color_thresh(birds_eye)
    # Plot the result
    #visualize_results(img, binary_color, pname, 'Combined Color Thresh.%s'%idx)

### Exploring Gradient Thresholds

In [ ]:
# Sobel
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

In [ ]:
# Test abs_sobel_thresh
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    sob_grad_binary = abs_sobel_thresh(test_image, orient='x', thresh_min=20, thresh_max=100)
    # Plot the result
    #visualize_results(test_image, sob_grad_binary, 'Original Image', 'Sobel Gradient')

In [ ]:
# Magnitude 
# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

In [ ]:
# Test mag_thresh
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    mag_binary = mag_thresh(test_image, sobel_kernel=3, mag_thresh=(30, 100))
    # Plot the result
    #visualize_results(test_image, mag_binary, 'Original Image', 'Magnitude Gradient')

In [ ]:
# Direction 
# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    # Grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

In [ ]:
# Test dir_threshold
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    dir_binary = dir_threshold(test_image, sobel_kernel=15, thresh=(0.7, 1.3))
    # Plot the result
    #visualize_results(test_image, dir_binary, 'Original Image', 'Direction Gradient')

In [ ]:
# Combined gradient and threshold
def comb_grad_thresh(img, ksize=3, sk_size=15, sx_thresh=(0, 255)):
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(img, orient='x', thresh_min=sx_thresh[0], thresh_max=sx_thresh[1])
    grady = abs_sobel_thresh(img, orient='y', thresh_min=sx_thresh[0], thresh_max=sx_thresh[1])
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(img, sobel_kernel=sk_size, thresh=(0.7, 1.3))
    combined_binary = np.zeros_like(dir_binary)
    combined_binary[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined_binary

In [ ]:
# Test combined gradient threshold
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    comb_binary = comb_grad_thresh(test_image, sx_thresh=(20, 100))
    # Plot the result
    #visualize_results(test_image, comb_binary, 'Original Image', 'Combined Gradient Thresh.')

In [ ]:
# Combined Color and Gradient Threshold
def color_grad_thresh(birds_eye,sx_thresh=(20, 100)):
    binary_color = comb_color_thresh(birds_eye)
    color_grad_binary = comb_grad_thresh(binary_color, sx_thresh=sx_thresh)
    return color_grad_binary

In [ ]:
# Test combined gradient threshold
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the function
    birds_eye, M, Minv = corners_unwarp(img)
    color_grad_binary = color_grad_thresh(birds_eye, sx_thresh=(20, 100))
    # Plot the result
    #visualize_results(test_image, color_grad_binary, 'Original Image', 'Combined Color and Gradient Thresh.')

### Step-5: Detect lane pixels and fit to find the lane boundary.

In [ ]:
# Implement Sliding Windows and Fit a Polynomial
def sliding_window_fit_poly(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)

    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    rect = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high), (0,255,0), 2) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high), (0,255,0), 2) 
        
        rect.append((win_xleft_low,
                    win_y_low, 
                    win_xleft_high,
                    win_y_high, 
                    win_xright_low, 
                    win_y_low, 
                    win_xright_high, 
                    win_y_high))
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Save the data for later use
    slide_pickle = {}
    slide_pickle["left_fit"] = left_fit
    slide_pickle["right_fit"] = right_fit
    pickle.dump(slide_pickle, open("output_images/wide_slide_pickle.p", "wb" ) )
    
    #print("Sliding window poly fit is done (one time only)")
    return left_fit, right_fit, left_lane_inds, right_lane_inds, leftx , rightx, lefty, righty, rect

### Visualization

In [ ]:
def visualize_sliding_fit(binary_warped, left_fit, right_fit, left_lane_inds, right_lane_inds, rect):   
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2] # left line
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] # right line
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0] # left pixels
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255] # right pixels
    
    for r in rect: 
        cv2.rectangle(out_img,(r[0],r[1]),(r[2],r[3]),(0,255,0), 2) 
        cv2.rectangle(out_img,(r[4],r[5]),(r[6],r[7]),(0,255,0), 2)
    
    plt.subplots(1, 1, figsize=(10,5))
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow') # left line
    plt.plot(right_fitx, ploty, color='yellow') # right line
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

In [ ]:
# visualize sliding fit
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions
    birds_eye, M, Minv = corners_unwarp(img)
    binary_warped = comb_color_thresh(birds_eye)
    # Plot the result
    #visualize_results(img, result_binary, pname, 'Color Thresh. Image%s'%idx)
    left_fit, right_fit, left_lane_inds, right_lane_inds, _,_,_,_,rect = sliding_window_fit_poly(binary_warped)
    # Plot the result
    visualize_sliding_fit(binary_warped, left_fit, right_fit, left_lane_inds, right_lane_inds, rect)

### Skip the sliding windows step once you know where the lines are
Now you know where the lines are you have a fit! In the next frame of video you don't need to do a blind search again, but instead you can just search in a margin around the previous line position like this:

In [ ]:
# Search in a margin around the previous line position
def prev_line_fit_poly(binary_warped, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_indsx = []
    right_lane_indsx = []
    
    left_lane_indsx = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_indsx = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_indsx]
    lefty = nonzeroy[left_lane_indsx] 
    rightx = nonzerox[right_lane_indsx]
    righty = nonzeroy[right_lane_indsx]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting 
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #print("Search lane in a margin around previously found line position is done...")
    return left_fitx , right_fitx, left_lane_indsx, right_lane_indsx, leftx, rightx, lefty, righty 

### Visualize

In [ ]:
def visualize_prev_line_fit(binary_warped, left_fitx, right_fitx, left_lane_indsx, right_lane_indsx):
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_indsx], nonzerox[left_lane_indsx]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_indsx], nonzerox[right_lane_indsx]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    ax , fig = plt.subplots(1, 1, figsize=(10,5))
    #print("axis dpi =",ax.dpi)
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)

In [ ]:
# visualize_prev_line_fit
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions

    birds_eye, M, Minv = corners_unwarp(img)
    binary_warped = comb_color_thresh(birds_eye)
    left_fitx , right_fitx, left_lane_indsx, right_lane_indsx, leftx, rightx, lefty, righty = prev_line_fit_poly(binary_warped, left_fit, right_fit)
    # Plot the result
    visualize_prev_line_fit(binary_warped, left_fitx, right_fitx, left_lane_indsx, right_lane_indsx)

### Step-6: Determine the radius of curvature and car offset

In [ ]:
# Measuring radius of curvature and car offset
# https://www.intmath.com/applications-differentiation/8-radius-curvature.php
def measure_curve(binary_warped, leftx, rightx, lefty, righty): 
    yvalue = image.shape[0]

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculate the new radii of curvature - real world (meters)
    left_curverad = ((1 + (2*left_fit_cr[0]*yvalue*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*yvalue*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
    
    # measure car offset
    lane_center = (left_fit_cr[-1] + right_fit_cr[-1])/2 # bottom of the image, position closest to the car
    car_camera_center = (binary_warped.shape[1] / 2) * xm_per_pix  # x-axis image midpoint
    car_offset = (car_camera_center - lane_center )# if +ve car on right of center of lane
    
    
    #print("Measuring curvature and caroffset is done...")
    
    return left_curverad, right_curverad, car_offset

In [ ]:
# test measure_curve 
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions
    birds_eye, M, Minv = corners_unwarp(img)
    binary_warped = comb_color_thresh(birds_eye)
   
    left_fitx , right_fitx, left_lane_indsx, right_lane_indsx , leftx , rightx, lefty, righty = prev_line_fit_poly(binary_warped, left_fit, right_fit)
    # Plot the result
    #visualize_prev_line_fit(result_binary, left_fitx, right_fitx, left_lane_indsx, right_lane_indsx)
    left_curverad, right_curverad, car_offset = measure_curve(binary_warped, leftx , rightx, lefty, righty)
    print(' %.2f' %left_curverad, ' %.2f' %right_curverad, ' %.2f' %car_offset)

### Step-7: Warp the detected lane boundaries back onto the original image.

In [ ]:
# warp lane boundaries on original image
def warp_lane(image, binary_warped, Minv, left_fitx , right_fitx):
    img = np.copy(image)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    cv2.polylines(color_warp, np.int_([pts_left]), isClosed=False, color=(255, 102, 255), thickness=25)
    cv2.polylines(color_warp, np.int_([pts_right]), isClosed=False, color=(255, 102, 255), thickness=25)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (binary_warped.shape[1], binary_warped.shape[0])) 
    # Combine the result with the original image
    newresult = cv2.addWeighted(img, 1, newwarp, 0.6, 0)
    #print("Wrap lane boundaries on original image is done...")
    return newresult

In [ ]:
# Test warp_lane
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions
    birds_eye, M, Minv = corners_unwarp(img)
    
    result_binary  = comb_color_thresh(birds_eye)

    left_fitx , right_fitx, left_lane_indsx, right_lane_indsx, leftx , rightx, lefty, righty = prev_line_fit_poly(result_binary, left_fit, right_fit)
    newresult = warp_lane(img, result_binary, Minv, left_fitx , right_fitx)
    # Plot the result
    #visualize_results(img, newresult, 'Test Image', 'Result Image')

### Step-8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
def display_position(image, left_curverad , right_curverad, car_offset):
    img = np.copy(image)

    if (car_offset >= 0):
        direction = 'right'
        text_d = 'Radius of Curvature = %.2f' %right_curverad + ' (m)'
    else:
        direction = 'left'
        text_d = 'Radius of Curvature = %.2f' %left_curverad + ' (m)'
    car_offset_abs = abs(car_offset)
    text_p = 'Vehicle is %.2f' %car_offset_abs + ' m' + ' %s' %direction + ' of center'
    
    org1 = (50, 50)
    org2 = (50, 100)
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (204, 255, 255)
    thickness = 2
    
    cv2.putText(img, text_d, org1, fontFace, fontScale, color, thickness)   
    cv2.putText(img, text_p, org2, fontFace, fontScale, color, thickness)
    #print("Output visual display is done...")
    return img

In [ ]:
# Test display_position
%matplotlib inline
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions
    birds_eye, M, Minv = corners_unwarp(img)
    result_binary  = comb_color_thresh(birds_eye)
   
    left_fitx , right_fitx, left_lane_indsx, right_lane_indsx, leftx , rightx, lefty, righty = prev_line_fit_poly(result_binary, left_fit, right_fit)
    left_curverad, right_curverad, car_offset = measure_curve(result_binary, leftx , rightx, lefty, righty)
    result_warp = warp_lane(img, result_binary, Minv, left_fitx , right_fitx)
    result_position = display_position(result_warp, left_curverad , right_curverad, car_offset )
    # Plot the result
    visualize_results(img, result_position, 'Test Image', 'Result Image')

In [ ]:
# detect lane pixels and fit to find the lane boundary 
    #slide_pickle = pickle.load(open("output_images/wide_slide_pickle.p" , "rb" ) )
    #left_fitc = slide_pickle["left_fit"]
    #right_fitc = slide_pickle["right_fit"]
    #left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc = prev_line_fit_poly(result_binaryc, left_fitc, right_fitc)

### Step-9: Line fit based on sanity checks

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        self.n = 5

    def write_fit(self, *args):
        # save last n values
        if len(self.recent_xfitted) <= self.n:
            self.recent_xfitted.append(args)
        else:
            del self.recent_xfitted[-1]
            self.recent_xfitted.append(args)
        
        
    def read_fit(self):
        # read last value
        #print("length=",len(self.recent_xfitted))
        fitx, lane_indsx, x, y = self.recent_xfitted[-1]
        return fitx, lane_indsx, x, y
            

In [ ]:
# Line fit based on sanity check - was a line detected in the last iteration?
def line_fit(left_line, right_line, result_binaryc):
    #print("line_fit")
    # initial
    left_fits, right_fits, left_lane_indss, right_lane_indss, leftxs , rightxs, leftys, rightys, _ = sliding_window_fit_poly(result_binary)
    # next
    left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc = prev_line_fit_poly(result_binaryc, left_fits, right_fits)      
    return left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc

In [ ]:
# Sanity check new fit results
#def sanity_check(leftxc , rightxc):
def sanity_check(leftxc , rightxc):
    sanity = False
    #diff is 700 +/- margin 
    diff = np.max(rightxc) - np.min(leftxc)
    if diff <= 950:
        sanity = True
    return sanity

In [ ]:
# Test sanity_check
left_line = Line()
right_line = Line()
left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc = line_fit(left_line, right_line, result_binary)  
print(np.max(rightxc))
print(np.min(leftxc))
diff = np.max(rightxc) - np.min(leftxc)
print(diff)
sanity = sanity_check(leftxc , rightxc)
print(sanity)

In [ ]:
# Write new good fit result
def write_good_fit(left_line, right_line, left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc): 
    #print("write_good_fit")
    left_line.write_fit(left_fitxc, left_lane_indsxc, leftxc, leftyc)
    right_line.write_fit(right_fitxc, right_lane_indsxc, rightxc, rightyc)

In [ ]:
# Read last good fit result
def read_good_fit(left_line, right_line):
    #print("read_good_fit")
    left_fitxc, left_lane_indsxc, leftxc, leftyc = left_line.read_fit()
    right_fitxc, right_lane_indsxc, rightxc, rightyc = right_line.read_fit()
    return left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc 

In [ ]:
# lane boundary fit with curve measurement
def lane_bound_fit(left_line, right_line, result_binaryc):
    #print("lane_bound_fit")
    left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc = line_fit(left_line, right_line, result_binaryc)    
    sanity = sanity_check(leftxc , rightxc)
    if sanity == True:
        write_good_fit(left_line, right_line, left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc)
        write_good_fit(left_line, right_line, left_fitxc , right_fitxc, left_lane_indsxc, right_lane_indsxc, leftxc , rightxc, leftyc, rightyc)
    #else:
        #print("Sanity Check - reading previous data")
    left_fitxcr , right_fitxcr, left_lane_indsxcr, right_lane_indsxcr, leftxcr , rightxcr, leftycr, rightycr = read_good_fit(left_line, right_line)
    left_curverad, right_curverad, car_offset = measure_curve(result_binaryc, leftxcr , rightxcr, leftycr, rightycr)
    return left_curverad, right_curverad, car_offset, left_fitxcr , right_fitxcr

In [ ]:
# Test lane_bound_fit
left_line = Line()
right_line = Line()
left_curverad, right_curverad, car_offset, left_fitxcr , right_fitxcr = lane_bound_fit(left_line, right_line, result_binary)
#print(left_fitxcr , right_fitxcr, left_lane_indsxcr, right_lane_indsxcr, leftxcr , rightxcr, leftycr, rightycr)
print(left_curverad, right_curverad, car_offset)

### Step-10: Integrate the pipeline and test on images

In [ ]:
# Integrating the advanced lane finding pipeline
def advanced_lane_finding(image):
    imgf = np.copy(image)   
    
    # perspective transform
    birds_eyef, Mf, Minvf = corners_unwarp(imgf)
    
    # color and/or gradient threshold
    result_binaryf = color_grad_thresh(birds_eyef, sx_thresh=(20, 100))
        
    # lane boundary fit 
    left_curveradf, right_curveradf, car_offsetf, left_fitxf , right_fitxf = lane_bound_fit(left_line, right_line, result_binaryf)    
    
    # warp lane boundaries on original image
    result_warpf = warp_lane(imgf, result_binaryf, Minvf, left_fitxf , right_fitxf)
    
    # output visual display 
    result_image = display_position(result_warpf, left_curveradf , right_curveradf, car_offsetf )
    
    #print("Advanced lane finding done!")

    return result_image


In [ ]:
left_line = Line()
right_line = Line()
# Test on all test images
t_images = sorted(glob.glob('output_images/output_test*'))
for idx, pname in enumerate(t_images):
    # Read images
    img = cv2.imread(pname)
    # Run the functions  
    #print(idx)
    result_image = advanced_lane_finding(img)
    # Plot the result
    visualize_results(img, result_image, 'Test Image', 'Result Image')

### Step-11: Test on videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
import os 
if not os.path.exists('test_videos_output'):
    os.mkdir('test_videos_output')

In [ ]:
# test with project_video.mp4
left_line = Line()
right_line = Line()
white_output = 'test_videos_output/project_video.mp4'
#clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,10)
clip1 = VideoFileClip("test_videos/project_video.mp4")
white_clip = clip1.fl_image(advanced_lane_finding) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
# test with project_video.mp4
white_output = 'test_videos_output/challenge_video.mp4'
clip2 = VideoFileClip("test_videos/challenge_video.mp4")
white_clip = clip2.fl_image(advanced_lane_finding) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
# test with project_video.mp4
white_output = 'test_videos_output/harder_challenge_video.mp4'
clip3 = VideoFileClip("test_videos/harder_challenge_video.mp4")
white_clip = clip3.fl_image(advanced_lane_finding) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)